In [1]:
import pandas as pd
import numpy as np
import boto3

In [2]:
path = 34
df = pd.read_csv(f'output/{path}_meteo_cleaned.csv', sep=';')
pd.set_option('display.max_columns', None)
df.head()

,POSTE,DATE,RR,DRR,TN,HTN,TX,HTX,TM,TMNX,TNSOL,TN50,TAMPLI,TNTXM,FFM,FXI,DXI,HXI,FXY,DXY,HXY,FXI3S,HXI3S,UN,HUN,UX,HUX,DHUMI40,DHUMI80,TSVM,UM,ORAG,BRUME,ETPMON,ETPGRILLE,Code INSEE,Code Postal,ville,geo_point_2d
0,34028003,2006-01-01,3.6,NaN,4.8,715.0,8.7,1315.0,6.9,6.75,NaN,NaN,3.9,6.8,6.8,23.0,300.0,2215.0,12.0,280.0,1730.0,NaN,NaN,63.0,1100.0,79.0,730.0,0.0,0.0,7.2,72.0,NaN,NaN,NaN,1.6,34028,34600,Bedarieux,"43.612556907446965, 3.16680787195388"
1,34028003,2006-01-02,0.0,NaN,4.5,115.0,7.9,1245.0,5.8,6.20,NaN,NaN,3.4,6.2,6.8,20.0,300.0,245.0,11.0,310.0,300.0,NaN,NaN,51.0,1300.0,87.0,130.0,0.0,125.0,6.3,68.0,NaN,NaN,NaN,2.1,34028,34600,Bedarieux,"43.612556907446965, 3.16680787195388"
2,34028003,2006-01-03,0.0,NaN,3.3,645.0,6.9,1145.0,4.3,5.10,NaN,NaN,3.6,5.1,6.6,20.0,340.0,145.0,10.0,310.0,100.0,NaN,NaN,52.0,1400.0,70.0,200.0,0.0,0.0,4.9,60.0,NaN,NaN,NaN,2.5,34028,34600,Bedarieux,"43.612556907446965, 3.16680787195388"
3,34028003,2006-01-04,0.0,NaN,1.6,1745.0,6.8,1315.0,2.8,4.20,NaN,NaN,5.2,4.2,4.3,15.0,320.0,45.0,7.0,330.0,430.0,NaN,NaN,49.0,1300.0,89.0,0.0,0.0,90.0,4.8,65.0,NaN,NaN,NaN,1.9,34028,34600,Bedarieux,"43.612556907446965, 3.16680787195388"
4,34028003,2006-01-05,2.4,NaN,-4.6,430.0,3.1,1145.0,-0.6,-0.75,NaN,NaN,7.7,-0.8,0.5,4.0,180.0,1215.0,2.0,360.0,30.0,NaN,NaN,71.0,1330.0,98.0,2130.0,0.0,1183.0,5.3,91.0,NaN,NaN,NaN,0.3,34028,34600,Bedarieux,"43.612556907446965, 3.16680787195388"


In [3]:
# fonction de moyenne lissante avec np.convolve
def moving_average(x, w):
    # Remplir le tableau d'entrée avec 'w//2' éléments de chaque côté en utilisant les valeurs de bord
    padded_x = np.pad(x, (w//2, w//2), mode='edge')
    # Effectuer la convolution avec le mode 'valid'
    return np.convolve(padded_x, np.ones(w), 'valid') / w



In [4]:
# ajout de colonne sur les précispitation moyenne par an et mois
df['moyenne precipitations année'] = moving_average(df['RR'], 365).round(2)
df['moyenne precipitations mois'] = moving_average(df['RR'], 31).round(2)

In [5]:
# moyenne ecapotranspiration par mois et année
df['moyenne evapotranspiration année'] = moving_average(df['ETPMON'], 365).round(2)
df['moyenne evapotranspiration mois'] = moving_average(df['ETPMON'], 31).round(2)

In [6]:
# moyenne vitesse de vent par mois et année
df['moyenne vitesse vent année'] = moving_average(df['FFM'], 365).round(2)
df['moyenne vitesse vent mois'] = moving_average(df['FFM'], 31).round(2)

In [7]:
# moyenne température par mois et année
df['moyenne temperature année'] = moving_average(df['TN'], 365).round(2)
df['moyenne temperature mois'] = moving_average(df['TN'], 31).round(2)

In [8]:
df.to_csv(f'output/{path}_meteo_cleaned_features.csv', sep=';', index=False, encoding='utf-8')

In [9]:
# print("Bucket utilisé :", bucket)
# s3 = boto3.client(
#     "s3",
#     aws_access_key_id=aws_access_key_id,
#     aws_secret_access_key=aws_secret_access_key,
#     region_name=region
# )
# s3.upload_file("corse_meteo_cleaned_insee_features.csv", bucket, "corse_meteo_cleaned_insee_features.csv")